# Chapter 20 Deep Generative Models

* 유주원 
<br><br>

# Chapter 20.5 Boltzmann Machines for Real-Valued Data

* 볼츠만 머신은 원래 바이너리 데이터와 함께 사용하기 위해 개발되었지만 이미지 및 오디오 모델링과 같은 많은 애플리케이션은 실제 값보다 확률 분포를 나타낼 수 있어야 한다.
* 경우에 따라 [0, 1] 구간의 실수 값 데이터를 이진 변수의 기대치로 나타낼 수 있다.
* 예를 들어, Hinton (2000)은 훈련 세트의 회색 음영 이미지를 [0,1] 확률 값으로 정의하여 취급한다.
* 각 픽셀은 이진 값이 1이 될 확률을 정의하고 이진 픽셀은 모두 서로 독립적으로 샘플링되며, 이는 그레이 스케일 이미지 데이터 세트에서 바이너리 모델을 평가하는 일반적인 절차이다.
* 그러나 이론적으로는 만족스러운 접근 방식은 아니며 이러한 방식으로 독립적으로 샘플링 된 이진 이미지는 노이즈가 있다.
* 이 섹션에서는 실제 값 데이터에 대한 확률 밀도를 정의하는 볼츠만 머신을 제시한다.

#### 20.5.1 Gaussian-Bernoulli RBMs

* restricted Boltzmann machine은 많은 지수적 조건부 분포에 의해 개발 될 수 있다.( Welling et al., 2005)
* 이 중 binary hidden units과 실수 값이 있는 visible units, 그리고 가우시안 분포를 따르는 조건부 분포를 가진 RBM이 가장 일반적이다.
* Gaussian-Bernoulli RBM을 파라미터화 하는 많은 방법이 있다.
* 우선 가우시안 분포에 대해 공분산 행렬 또는 precision matrix를 사용할지 여부를 선택할 수 있다. (precision matrix : 공분산의 역행렬)
* 우리는 아래와 같은 조건부 분포를 얻기를 희망한다.
<img src="./figures/fig1.png" />
* 비정규화된 로그 조건부 분포를 확장함으로써 에너지 함수를 추가해야 할 필요가 있다.
<img src="./figures/fig2.png" />

* 여기서 f는 모델의 임의의 변수가 아닌 매개 변수의 함수인 모든 항을 캡슐화한다.
* 분포를 정규화하기 위해 우리는 f를 버릴 수가 있으며, 우리가 선택한 모든 energy function의 partition function이 그 역할을 수행할 것이다.
* 위의 식에는 아래의 항이 포함되어 있다.
<img src="./figures/fig3.png" />

* 이 항에는 $h_ih_j$ 항이 포함되어 있기 때문에 전체적으로 포함될 수 없다.(This term cannot be included in its entirety because it includes $h_ih_j$ terms.) 
* 이것은 hidden unit 사이의 edge에 해당된다.
* 이 조건을 포함시키면 restricted Boltzmann machine 대신 linear factor model을 가지게 된다.
* Boltzmann machine을 설계할 때, 우리는 단순히 이러한 $h_ih_j$ 교차 항을 생략한다.
* 이들을 생략해도 조건부 p(v|h)는 변하지 않는다.

<img src="./figures/fig4.png" />

* 위의 경우 h1 ∈ {0, 1}이기 때문에, $h^2_i = hi$ 를 사용한다.
* 만약 이 term이 에너지 함수에 포함이 된다면, unit에 대한 가중치가 커질 때, 자연스럽게 $h_i$가 turn off 되기 위해 편향될 것이고, 높은 precision을 가진 visible unit에 연결될 것이다.
* 이 bias term을 포함할지 여부는 모델이 나타낼 수 있는 분포 계열에는 영향을 미치지 않지만 모델의 학습에는 영향을 미친다.
* 이 term을 포함하면 가중치가 급격히 증가하더라도 hidden unit activation이 적절하게 유지되는 데 도움이 될 수 있다.
* Gaussian-Bernoulli RBM에서 아래의 방법으로 에너지 함수를 정의할 수 있다.
<img src="./figures/fig5.png" />

* Gaussian-Bernoulli RBM의 매개변수의 가변성의 한가지 원인은 precision matrix를 처리하는 방법을 선택하는 것이다.
* 상수로 고정되거나 학습 된 것일 수도 있다. 또한 단위 행렬의 스칼라 곱일 수도 있고 대각 행렬이 될 수도 있다.
* 일반적으로 precision matrix를 non-diagonal이 되도록 허용하지 않는다. 왜냐하면 어떤 연산의 경우 matrix의 역을 요구할 수 있기 때문이다.
* 앞으로의 섹션에서 우리는 다른 형태의 볼츠만 머신이 precision matrix의 역을 피하기 위해 다양한 기법을 사용하여 공분산 구조를 모델링 할 수 있음을 배우게 될 것이다.

#### 20.5.2 Undirected Models of Conditional Covariance

* Gaussian RBM은 실수 값 데이터에 대한 표준 에너지 모델이었지만, Ranzato et al. (2010a)는 Gaussian RBM 유도 바이어스가 실제 데이터의 일부 유형, 특히 자연 이미지에 존재하는 통계적 변형에 적합하지 않다고 주장했다.
* 문제는 자연 이미지에 있는 많은 정보 내용이 원시 픽셀 값이 아닌 픽셀 간의 공분산에 포함된다는 것이다.
* 즉, 이미지의 유용한 정보가 있는 것은 절대 값이 아니라 픽셀 간의 관계이다.
* Gaussian RBM은 hidden unit이 주어진 경우 입력의 조건부 평균만 모델링 하기 때문에 조건부 공분산 정보를 포착할 수 없다.
* 이러한 문제에 대응하여 실제 값 데이터의 공분산을 더 잘 설명하려는 대체 모델이 제안되었다.
* 이러한 모델에는 평균과 공분산 RBM(mcRBM1), t분포(mPoT) 모델의 평균, spike와 slab RBM(ssRBM)등이 있다.

##### Mean and Covariance RBM
* mcRBM은 hidden unit을 사용해서 관찰된 모든 유닛의 조건부 평균 및 공분산을 독립적으로 인코딩한다.
* mcRBM의 hidden unit은 두 개의 단위 그룹으로 나뉘는데 평균 단위와 공분산 단위이다.
* 조건부 평균을 모델링하는 그룹은 단순히 Gaussian RBM이다.
* 나머지 반은 cRBM이라 불리는 공분산 RBM(Ranzato et al., 2010a)이며, 그 구성 요소는 아래에 설명된 조건부 공분산 구조를 모델링한다.
* 특히 이진 평균 단위 h(m) 및 이진 공분산 단위 h(c)를 가지는 mcRBM 모델은 두 가지 에너지 함수의 조합으로 정의된다.
<img src="./figures/fig6.png" />

* $E_m$은 표준 Gaussian-Bernoulli RBM 에너지 함수이다.
<img src="./figures/fig7.png" />

* $E_c$는 cRBM 에너지 함수이며 모델의 조건부 공분산 정보를 나타낸다.
<img src="./figures/fig8.png" />

* parameter $r^{(j)}$는 $h^{(c)}_j$와 관련된 공분산 가중치 벡터와 일치하고 $b^{(c)}$는 공분산 offset의 벡터이다.
* 결합된 에너지 함수는 joint distribution으로 정의된다.
<img src="./figures/fig9.png" />

$h^{(m)}$과 $h^{(c)}$에 주어진 관측치에 상응하는 조건부 분포를 다변량 가우시안 분포로 나타내면 다음과 같다.
<img src="./figures/fig10.png" />
<img src="./figures/fig11.png" />

* 공분산 행렬 $C^{mc}_{x|h}$는 non-diagonal이며, W는 조건부 평균을 모델링하는 가우시안 RBM과 관련된 가중치 행렬이다.
* non-diagonal 조건부 공분산 구조 때문에 contrastive divergence 또는 persistent contrastive divergence를 통한 mcRBM 학습이 어렵다.
* CD와 PCD는 x, $h^{(m)}$, $h^{(c)}$의 공동 분포로부터 표본 추출을 요청하며, 표준 RBM에서는 조건에 대한 깁스 샘플링에 의해 성취된다.
* 그러나 mcRBM에서 $p_{mc}(x|h^{(m)}, h^{(c)})$ 로부터의 표본 추출은 학습때마다 $(C^{mc}) -1$ 의 계산을 요구한다.
* 이것은 더 큰 관측에 대해 비실용적인 계산이 될 수 있다.
* Ranzato and Hinton(2010)은 mcRBM 자유 에너지 상의 Hamiltonian(hybrid) Monte Carlo (Neal, 1993)을 사용해서 margianl p(x)로부터 직접 샘플링함으로써 조건부 $pmc(x|h^{(m)}, h^{(c)})$로부터 직접 샘플링하는 것을 피했다.

##### Mean-Product of Student's t-distributions
* mean-product of Student's t-distribution(mPot) 모델 (Ranzato et al., 2010b)은 mcRBM이 cRBM을 확장하는 것과 유사한 방식으로 PoT 모델(Welling et al., 2003a)을 확장했다. 
* 이는 Gaussian RBM과 유사한 hidden unit을 추가함으로써 Gaussian 평균이 0을 포함하지 않도록 만든다.
* mcRBM과 마찬가지로 관측치에 대한 PoT 조건부 분포는 다변수 가우스 분포이다.
* 그러나 mcRBM과 달리 hidden 변수에 대한 조건부 분포는 조건부 독립적인 감마 분포에 의해 주어진다.
* 감마 분포 G(k,θ) 는 양의 실수에 대한 확률 분포이며 평균은 kθ이다.
* mPoT 모델의 기본 아이디어를 이해하기 위해 감마 분포를 더 자세히 이해할 필요는 없다.
* mPoT 에너지 함수는 다음과 같다.
<img src="./figures/fig12.png" />

* $r^{(j)}$는 $h^{(c)}_j$와 $E_m(x,h^{(m)})$과 관련된 공분산 가중치 벡터이다.
* mcRBM에서와 마찬가지로, mPoT 모델 에너지 함수는 non-diagonal 공분산을 가지는 x상의 조건부 분포를 가지는 다변수 가우시안을 명시한다.
* mPoT 모델에서의 학습은 non-diagonal 가우시안 조건부 $p_{mPoT}(x|h^{(m)}, h^{(c)})$로부터 샘플링할 수 없기 때문에 해밀턴 몬테카를로를 통해 p(x)를 직접 샘플링 한다. 

##### Spike and Slab Restricted Boltzmann Machines
* spike and slab restricted Boltzmann machines 또는 ssRBMs는 실수 값 데이터의 공분산 구조를 모델링하는 또 다른 방법을 제공한다.
* mcRBM과 비교할 때, ssRBM은 역행렬과 해밀턴 몬테 카를로 방법을 필요로 하지 않는다는 장점이 있다.
* ssRBM은 두 종류의 hidden units이 있다. 바로 binary spike unit h와 실수 값 slab unit s이다.
* visible unit의 평균은 아래와 같이 주어진다.
<img src="./figures/fig13.png" />

* 즉, 각각의 $columnW_{:,i}$는 $h_i = 1$일 때, 입력에 나타날 수 있는 구성요소를 정의한다.
* 대응되는 spike 변수 $h_i$는 그 구성요소가 존재하는지 여부를 결정한다.
* slab 변수 $s_i$는 해당 구성 요소의 강도를 결정한다.
* spike 변수가 활성화 되면, 일치하는 slab 변수는 $W_{:,i}$에 정의된 축을 따라 입력에 variance를 추가한다.
* 이를 통해 입력의 공분산을 모델링 할 수 있다.
* 다행히도 깁스 샘플링을 이용한 contrastive divergence와 persistent contrastive divergence가 여전히 적용 가능하다.
* 공식적으로 ssRBM 모델은 에너지 함수를 통해 정의된다.
<img src="./figures/fig14.png" />

* $b_i$는 spike $h_i$의 offset이고 Λ는 observations x 상의 diagonal precision matrix이다.
* 파라미터 $α_i > 0$ 는 실수값 slab 변수 $s_i$에 대한 scalar precision 파라미터이다.
* 파라미터 $Φ_i$는 non-negative diagonal matrix이며, 각 $μ_i$는 slab 변수 $s_i$에 대한 평균 파라미터이다.
* binary spike 변수 h가 주어진 observation 상에서 조건부 분포는 다음과 같다.
<img src="./figures/fig15.png" />
<img src="./figures/fig16.png" />
* 위의 식은 공분산 행렬 $cs^{ss}_{x|h}$ 가 양수일 경우에만 성립한다.

* mcRBM과 mPoT, ssRBM의 차이를 설명하자면 우선 mcRBM과 mPoT는 조건부 공분산에 대한 제약을 걸기 위해 hidden unit $h_j > 0$ 의 activation을 사용하여 observation의 공분산 구조를 모델링한다.
* 반면 ssRBM은 hidden spike activations $h_i = 1$을 사용함으로써 observation의 조건부 공분산을 명시한다.
* ssRBM의 주요 단점은 파라미터의 일부 설정이 양의 값이 아닌 공분산 행렬에 해당할 수 있다는 것이다.
* 이러한 공분산 행렬은 unnormalize된 확률을 평균으로부터 더 멀리 있게 해서 모든 가능한 결과에 대한 적분을 벗어나게 한다.
* 일반적으로 간단한 휴리스틱 트릭을 사용하여 이 문제를 피할 수 있지만 이론적으로 만족할만한 해결책은 없다.
* ssRBM은 몇 가지 확장을 허용한다.
* 고차원의 interaction과 slab variable의 average-pooling을 포함하면 labeled data가 부족할 때, 모델이 분류기의 뛰어난 특징을 학습하는 것이 가능하다.